# Bornoulli coin tossing
http://pyro.ai/examples/svi_part_i.html

In [1]:
import math
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

from pyro.infer.autoguide import AutoDiagonalNormal

ModuleNotFoundError: No module named 'torch'

In [2]:
# Pyro stores some info between computations, we need to clear that
pyro.clear_param_store()

NameError: name 'pyro' is not defined

In [3]:
# create a dataset of 10 tails and 20 heads
data = []

tails_number = 10
heads_number = 20

for _ in range(tails_number):
    data.append(torch.tensor(1.0))
for _ in range(heads_number):
    data.append(torch.tensor(0.0))
print(data)

[tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.)]


We will use the Bete distribution with parameters $\alpha$ and $\beta$ as a prior for the parameter of our Bernoulli random variable.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Beta_distribution_pdf.svg/488px-Beta_distribution_pdf.svg.png' style='background:white' />

In [4]:
def model(data):
    """
    Generative model: 
    * sample Bernoulli parameter from the prior Beta distribution
    * 'toss' a coin with sampled parameter of heads, mark as observed and plug in data
    """  
    # Prior
    alpha_zero = torch.tensor(10.0)
    beta_zero = torch.tensor(10.0)
    # Sample heads probability
    fairness = pyro.sample("latent_fairness", dist.Beta(alpha_zero, beta_zero))

    # Each toss is independent with the same probability
    for i in range(len(data)):
        # Provide observed values
        pyro.sample("obs_{}".format(i), dist.Bernoulli(fairness), obs=data[i])

In [5]:
def guide(data):
    # Posterior distribution will be in the same family: Beta
    # Initial values of the parameters will be equal
    alpha_q = pyro.param("alpha_q", torch.tensor(20.0),
                        constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(20.0),
                      constraint=constraints.positive)
    # Sample from approximate posterior
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

In [6]:
# setup the optimizer
adam_parameters = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_parameters)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

In [7]:
pyro.clear_param_store()

step_number = 1900

for j in range(step_number):
    # calculate the loss and take a gradient step
    loss = svi.step(data)
    if j % 100 == 0:
        print("[iteration {:} / {:}] loss: {:.4}".format(j + 1, step_number, loss))

[iteration 1 / 1900] loss: 22.24
[iteration 101 / 1900] loss: 22.76
[iteration 201 / 1900] loss: 19.37
[iteration 301 / 1900] loss: 20.12
[iteration 401 / 1900] loss: 20.32
[iteration 501 / 1900] loss: 19.74
[iteration 601 / 1900] loss: 19.94
[iteration 701 / 1900] loss: 20.38
[iteration 801 / 1900] loss: 20.36
[iteration 901 / 1900] loss: 20.16
[iteration 1001 / 1900] loss: 20.31
[iteration 1101 / 1900] loss: 20.11
[iteration 1201 / 1900] loss: 20.35
[iteration 1301 / 1900] loss: 20.44
[iteration 1401 / 1900] loss: 20.13
[iteration 1501 / 1900] loss: 20.19
[iteration 1601 / 1900] loss: 20.5
[iteration 1701 / 1900] loss: 20.46
[iteration 1801 / 1900] loss: 20.27


Extract posterior parameters:

In [8]:
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

Here we use some facts about the beta distribution:
* compute the inferred mean of the coin's fairness
* compute inferred standard deviation

In [10]:
inferred_mean = alpha_q / (alpha_q + beta_q)
inferred_std = math.sqrt(alpha_q * beta_q / (alpha_q + beta_q)**2 / (1.0 + alpha_q + beta_q))

print(f"Based on the data and our prior belief, the fairness of the coin is {inferred_mean:.3f} +- {inferred_std:.3f}")

Based on the data and our prior belief, the fairness of the coin is 0.400 +- 0.076
